# Data prep

## In this notebook we will load the songs from the zip file, and perform transoformations to prepare the data for two-tower training
Steps
1. Extract from the zip file
2. Upload to BQ
3. Enrich features for the playlist songs
4. Cross-join songs with features (excpected rows = n_songs x n_playlists)
5. Remove after-the-fact (later position songs) from the newly generated samples
6. Create a clean train table, and flatten structs or use arrays

#### Unzip the file and uplaod to BQ

In [ ]:
# !gsutil cp gs://matching-engine-content/spotify-million-playlist/spotify_million_playlist_dataset.zip .
# !unzip spotify_million_playlist_dataset.zip

In [ ]:
# import os
# import json

# data_files = os.listdir('data')
# # PROJECT_ID = 'jtotten-project'
# bq_dataset = 'mdp_eda'

# for filename in data_files:
#   with open(f'data/{filename}') as f:
#     print(f)
#     json_dict = json.load(f)
#     df = pd.DataFrame(json_dict['playlists'])
#     df.to_gbq(
#     destination_table=f'{bq_dataset}.playlists', 
#     project_id=f'{PROJECT_ID}', # TODO: param
#     location='us-central1', 
#     progress_bar=True, 
#     reauth=True, 
#     if_exists='append'
#     ) 

### Need to find playlist_nested lineage

Here's the ddl from the schema - could be created from playlists table below

```
'CREATE TABLE `hybrid-vertex.spotify_mpd.playlists_nested`\n(\n  name STRING,\n  collaborative STRING,\n  pid INT64,\n  modified_at INT64,\n  num_tracks INT64,\n  num_albums INT64,\n  num_followers INT64,\n  num_edits INT64,\n  duration_ms INT64,\n  num_artists INT64,\n  description STRING,\n  tracks ARRAY<STRUCT<pos STRING, artist_name STRING, track_uri STRING, artist_uri STRING, track_name STRING, album_uri STRING, duration_ms STRING, album_name STRING>>,\n  row_number INT64\n);'
```

In [ ]:
# %%bigquery
# create or replace table jtotten-project.spotify_mpd.playlists as (
# with json_parsed as (SELECT *, JSON_EXTRACT_ARRAY(tracks) as json_data FROM `jtotten-project.mdp_eda.playlists` )

# select json_parsed.* except(tracks, duration_ms, json_data),
# JSON_EXTRACT(jsn_line, "$.pos") as pos, 
# JSON_EXTRACT(jsn_line, "$.artist_name") as artist_name,
# JSON_EXTRACT(jsn_line, "$.track_uri") as track_uri,
# JSON_EXTRACT(jsn_line, "$.artist_uri") as artist_uri,
# JSON_EXTRACT(jsn_line, "$.track_name") as track_name,
# JSON_EXTRACT(jsn_line, "$.album_uri") as album_uri,
# JSON_EXTRACT(jsn_line, "$.duration_ms") as duration_ms,
# JSON_EXTRACT(jsn_line, "$.album_name") as album_name,

# from json_parsed CROSS JOIN UNNEST(json_parsed.json_data) as jsn_line
# )

## Now enrich the playlist songs with the new features

`hybrid-vertex.spotify_train_3.unique_track_features` # need code

+

`hybrid-vertex.spotify_train_3.unique_artist_features` # need code

+

`hybrid-vertex.spotify_mpd.playlists_nested`

Trainv3-create-enriched-data


In [5]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.enriched_data` AS (
    SELECT
    a.* except(tracks),
      ARRAY(
    SELECT
      AS STRUCT CAST(track.pos AS int64) AS pos_can,
      track.artist_name AS artist_name_can,
      track.track_uri AS track_uri_can,
      track.track_name AS track_name_can,
      track.album_uri AS album_uri_can,
      track.artist_uri AS artist_uri_can,
      CAST(track.duration_ms AS float64) AS duration_ms_can,
      track.album_name AS album_name_can,
      tf.track_pop AS track_pop_can,
      af.artist_pop AS artist_pop_can,
      af.artist_genres AS artist_genres_can,
      af.artist_followers AS artist_followers_can
    FROM
      UNNEST(tracks) as track
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_track_features` AS tf --track features
    ON
      (track.track_uri = tf.track_uri)
    INNER JOIN
      `hybrid-vertex.spotify_train_3.unique_artist_features` AS af --artist features
      ON
      (track.artist_uri = af.artist_uri)
      ) AS tracks
  FROM 
  `hybrid-vertex.spotify_mpd.playlists_nested` as a)

Query complete after 0.00s: 100%|██████████| 38/38 [00:00<00:00, 21353.64query/s]                      


""


## Cross join + get rid of after-the-fact pos data in playlist

cross_join_songxplaylist_struct_query

`hybrid-vertex.spotify_train_3.ordered_position_training`



In [ ]:
pos_seed
artist_name_can
track_uri_can
track_name_can
album_uri_can
artist_uri_can
duration_ms_can
album_name_can
track_pop_can
artist_pop_can
artist_genres_can
artist_followers_can

In [37]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.ordered_position_training` AS (
  WITH
    -- get every combination of song and its parent playlist
    unnest_cross AS(
    SELECT
      b.*,
      CONCAT(b.pid,"-",track.pos_can) AS pid_pos_id,
      CAST(track.pos_can AS int64) AS pos_can,
      track.artist_name_can ,
      track.track_uri_can ,
      track.album_uri_can,
      track.track_name_can ,
      track.artist_uri_can ,
      CAST(track.duration_ms_can AS float64) AS duration_ms_can,
      track.album_name_can ,
      track.track_pop_can ,
      track.artist_pop_can,
      track.artist_genres_can ,
      track.artist_followers_can 
    FROM (
      SELECT
        * EXCEPT(duration_ms),
        duration_ms AS duration_ms_playlist
      FROM
        `hybrid-vertex.spotify_train_3.enriched_data`) AS b
    CROSS JOIN
      UNNEST(tracks) AS track)
  SELECT
    a.* EXCEPT(tracks,
      num_tracks,
      num_artists,
      num_albums,
      num_followers,
      num_edits),
    ARRAY(
    SELECT
      AS STRUCT CAST(track.pos_can AS int64) AS pos_pl,
      track.artist_name_can AS artist_name_pl,
      track.track_uri_can AS track_uri_pl,
      track.track_name_can AS track_name_pl,
      track.album_uri_can AS album_uri_pl,
      track.artist_uri_can AS artist_uri_pl,
      CAST(track.duration_ms_can AS float64) AS duration_ms_pl,
      track.album_name_can AS album_name_pl,
      track.track_pop_can AS track_pop_pl,
      track.artist_pop_can AS artist_pop_pl,
      track.artist_genres_can AS artist_genres_pl,
      track.artist_followers_can AS artist_followers_pl,
      # track_pop_can AS track_pop_pl,
      # artist_pop_can AS artist_pop_pl,
      # artist_genres_can AS artist_genres_pl,
      # artist_followers_can AS artist_followers_pl
    FROM
      UNNEST(tracks) AS track
    WHERE
      CAST(track.pos_can AS int64) < a.pos_can ORDER BY CAST(track.pos_can AS int64)) AS seed_playlist_tracks,
    ----- seed track part
    trx.pos_can AS pos_seed_track,
    trx.artist_name_can AS artist_name_seed_track,
    trx.artist_uri_can AS artist_uri_seed_track,
    trx.track_name_can AS track_name_seed_track,
    trx.track_uri_can AS track_uri_seed_track,
    trx.album_name_can AS album_name_seed_track,
    trx.album_uri_can AS album_uri_seed_track,
    trx.duration_ms_can AS duration_seed_track,
    trx.track_pop_can AS track_pop_seed_track,
    trx.artist_pop_can AS artist_pop_seed_track,
    trx.artist_genres_can as artist_genres_seed_track,
    trx.artist_followers_can as artist_followers_seed_track
  FROM
    unnest_cross AS a -- with statement
    ,
    UNNEST(a.tracks) AS trx
  WHERE
    CAST(trx.pos_can AS int64) = a.pos_can-1);
  -- LIMIT
  -- 100;

Query complete after 0.00s: 100%|██████████| 4/4 [00:00<00:00, 1804.00query/s]                        


""


## Update the playlist metadata with the new samples created above

Trainv3-clean-track-features

Get new metadata for the tracks now that there are updated track counts, durations, etc...

`hybrid-vertex.spotify_train_3.train`


In [38]:
%%bigquery
create or replace table `hybrid-vertex.spotify_train_3.train` as (
WITH
  playlist_features_clean AS (
  SELECT
    pid_pos_id,
    SUM(trx.duration_ms_pl) AS duration_ms_seed_pl,
    COUNT(1) AS n_songs_pl,
    COUNT(DISTINCT trx.artist_name_pl) AS num_artists_pl,
    COUNT(DISTINCT trx.album_uri_pl) AS num_albums_pl,
  FROM
    `hybrid-vertex.spotify_train_3.ordered_position_training`,
    UNNEST(seed_playlist_tracks) AS trx
  GROUP BY
    pid_pos_id)
SELECT
  a.* except(artist_genres_can, artist_genres_seed_track),
  b.* except(pid_pos_id),
  a.artist_genres_can[OFFSET(0)] as artist_genres_can,
  a.artist_genres_seed_track[OFFSET(0)] as artist_genres_seed_track,
  # unnested.artist_genres_pl[OFFSET(0)] as artist_genres_pl
FROM
  `hybrid-vertex.spotify_train_3.ordered_position_training` a
INNER JOIN
  playlist_features_clean b
ON
  a.pid_pos_id = b.pid_pos_id )#, UNNEST(seed_playlist_tracks) as unnested)

Query complete after 0.00s: 100%|██████████| 34/34 [00:00<00:00, 14828.57query/s]                      


""


## get rid of structs by createing new table with arrays from playlist_seed


In [39]:
%%bigquery
create or replace table `hybrid-vertex.spotify_train_3.train_flatten` as (
SELECT a.* except(seed_playlist_tracks, description),
    IFNULL(a.description, "") as description_pl,
    ARRAY(select t.pos_pl from UNNEST(seed_playlist_tracks) t) as pos_pl,
    ARRAY(select t.artist_name_pl from UNNEST(seed_playlist_tracks) t) as artist_name_pl,
    ARRAY(select t.track_uri_pl from UNNEST(seed_playlist_tracks) t) as track_uri_pl,
    ARRAY(select t.track_name_pl from UNNEST(seed_playlist_tracks) t) as track_name_pl,
    ARRAY(select t.duration_ms_pl from UNNEST(seed_playlist_tracks) t) as duration_ms_songs_pl,
    ARRAY(select t.album_name_pl from UNNEST(seed_playlist_tracks) t) as album_name_pl,
    ARRAY(select t.artist_pop_pl from UNNEST(seed_playlist_tracks) t) as artist_pop_pl,
    ARRAY(select t.artist_followers_pl from UNNEST(seed_playlist_tracks) t) as artists_followers_pl,
    ARRAY(select case when t.track_pop_pl is null then 0 else t.track_pop_pl end from UNNEST(seed_playlist_tracks) t) as track_pop_pl,
    ARRAY(select t.artist_genres_pl[OFFSET(0)] from UNNEST(seed_playlist_tracks) t) as artist_genres_pl
    from `hybrid-vertex.spotify_train_3.train` a
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1637.33query/s]                        


""


## We will use this table to export jsonl in the next notebook

In [ ]:
#tryin this bc  (1) Invalid argument: no value for /query/duration_ms_playlist

In [53]:
%%bigquery
CREATE OR REPLACE TABLE
  `hybrid-vertex.spotify_train_3.train_json_export_table` AS (
  SELECT
    STRUCT(
        ARRAY(select artist_name_can) as artist_name_can,
        ARRAY(select track_uri_can) as track_uri_can,
        # album_uri_can,
        ARRAY(select track_name_can) as track_name_can,
        # artist_uri_can,
        ARRAY(select duration_ms_can) as duration_ms_can,
        ARRAY(select album_name_can) as album_name_can,
        ARRAY(select case when track_pop_can is null then 0. else cast(track_pop_can as FLOAT64) end) as track_pop_can,
        ARRAY(select case when artist_pop_can is null then 0. else artist_pop_can end) as artist_pop_can,
        ARRAY(select case when artist_followers_can is null then 0. else artist_followers_can end) as artist_followers_can,
        ARRAY(select artist_genres_can) as artist_genres_can) AS candidate,
    STRUCT(ARRAY(select pid_pos_id) as id_pl,
        ARRAY(select name) as name,
        ARRAY(select collaborative) as collaborative,
        ARRAY(select artist_name_seed_track) as artist_name_seed_track,
        # artist_uri_seed_track,
        ARRAY(select track_name_seed_track) as track_name_seed_track,
        # track_uri_seed_track,
        ARRAY(select album_name_seed_track) as album_name_seed_track,
        # album_uri_seed_track,
        ARRAY(select duration_seed_track) as duration_seed_track,
        ARRAY(select case when track_pop_seed_track is null then 0.0 else cast(track_pop_seed_track as FLOAT64) end) as track_pop_seed_track,
        ARRAY(select case when artist_pop_seed_track is null then 0.0 else artist_pop_seed_track end) as artist_pop_seed_track,
        ARRAY(select artist_followers_seed_track) as artist_followers_seed_track,
        ARRAY(select duration_ms_seed_pl) as duration_ms_seed_pl,
        ARRAY(select cast(n_songs_pl as FLOAT64)) as n_songs_pl,
        ARRAY(select cast(num_artists_pl as FLOAT64)) as num_artists_pl,
        ARRAY(select cast(num_albums_pl as FLOAT64)) as num_albums_pl,
        ARRAY(select artist_genres_seed_track) as artist_genres_seed_track,
        ARRAY(select description_pl) as description_pl,
        artist_name_pl,
        # track_uri_pl,
        track_name_pl,
        duration_ms_songs_pl,
        album_name_pl,
        cast(artist_pop_pl as ARRAY<FLOAT64>) as artist_pop_pl,
        artists_followers_pl,
        track_pop_pl,
        artist_genres_pl,
        duration_ms_playlist) AS query
  FROM
    `hybrid-vertex.spotify_train_3.train_flatten`)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1543.35query/s]                        


""
